# Geração de instâncias

In [4]:
import gurobipy as gp
env = gp.Env()

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2372514
Academic license - for non-commercial use only - registered to braun@cos.ufrj.br


In [6]:
from udgp import M4, generate_random_instance

# cicloexano: seed 144

N = 10
instance = generate_random_instance(N, 12345)

instance.view_input()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [9]:
m1 = M4(instance, n=4, max_gap=1e-4, log=False, env=env)
m1.optimize()

m1.instance.view()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [10]:
# m1 = M4(instance, n=4, max_gap=1e-2, log=False, env=env)
# m1.optimize()

instance.mock_core()

for i in range(instance.n - 5):
    m = M4(instance, n=1, max_gap=1e-2, log=False, env=env)
    m.optimize()
    instance = m.instance

instance.view()

Modelo inviável.


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol